In [1]:
from sklearn.cluster import BisectingKMeans as bkm
from sklearn.svm import SVC
import concurrent.futures
import pandas as pd
import numpy as np
import time
import sys

In [2]:
pd.__version__

'1.4.2'

In [431]:
import sklearn
sklearn.__version__

'1.1.1'

In [2]:
np.__version__

'1.24.4'

In [5]:
vectors=pd.read_csv('_totalVector06640.csv',sep='\t',index_col=0)
vectors=vectors.astype({'0':'category','1':'category',vectors.columns[-1]:'category'})
#pump_labels=np.ascontiguousarray(pd.read_csv('/home/act/Desktop/appspack/examples/1506/5_labelfile.csv',sep=',',index_col=0),dtype=float).flatten()

In [6]:
data=np.ascontiguousarray(vectors.iloc[:,3:-1],dtype=float)
labels=np.ascontiguousarray(vectors.iloc[:,-1],dtype=int)

In [344]:
np.std(data[0:5,:],axis=1)

array([1., 1., 1., 1., 1.])

In [153]:
nopumps=data[np.where(np.logical_not(labels))]

In [154]:
np.where(np.logical_not(labels))

(array([    0,     1,     2, ..., 55138, 55139, 55140], dtype=int64),)

In [8]:
nopumps.shape

(29587, 80)

In [6]:
data.shape

(55141, 80)

In [91]:
nopumps.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [21]:
for x in map(lambda params:(lambda i,y: (i**2,y))(*params),enumerate(range(10))):
    print(x)

(0, 0)
(1, 1)
(4, 2)
(9, 3)
(16, 4)
(25, 5)
(36, 6)
(49, 7)
(64, 8)
(81, 9)


In [133]:
list(map(lambda args:args[0],np.delete(np.transpose([np.arange(10),np.random.rand(10)]),np.array([3,6,7]),axis=0)))

[0.0, 1.0, 2.0, 4.0, 5.0, 8.0, 9.0]

In [132]:
np.delete(np.transpose([np.arange(10),np.random.rand(10)]),np.array([3,6,7]),axis=0)

array([[0.        , 0.64111522],
       [1.        , 0.37903055],
       [2.        , 0.13111926],
       [4.        , 0.8445464 ],
       [5.        , 0.45834884],
       [8.        , 0.97255675],
       [9.        , 0.69904165]])

In [ ]:
np.array([list(x) for x in enumerate(range(10))])

array([[0, 0],
       [1, 1],
       [2, 2],
       [3, 3],
       [4, 4],
       [5, 5],
       [6, 6],
       [7, 7],
       [8, 8],
       [9, 9]])

In [6]:
A=np.random.rand(4,5)
print(A)

[[0.48862115 0.15000663 0.98586891 0.71722833 0.46704862]
 [0.62046666 0.3281133  0.08962857 0.95452023 0.62005765]
 [0.69199038 0.98163673 0.83010851 0.95677889 0.99759847]
 [0.60632077 0.42066488 0.05687041 0.94658752 0.65007123]]


In [256]:
np.array([(1,2),(2,3)])

array([[1, 2],
       [2, 3]])

In [254]:
np.arange(2,10)

array([2, 3, 4, 5, 6, 7, 8, 9])

In [18]:
def get_index(i,j):
    return (i**2,2*j)
A[np.vectorize(get_index)(1,range(3))]=np.zeros(3)

In [53]:
np.vectorize(get_index)(range(3),range(3))

(array([0, 1, 4]), array([0, 2, 4]))

In [16]:
{i:i**2 for i in range(10)}[6]

36

In [19]:
A

array([[0.48862115, 0.15000663, 0.98586891, 0.71722833, 0.46704862],
       [0.        , 0.3281133 , 0.        , 0.95452023, 0.        ],
       [0.69199038, 0.98163673, 0.83010851, 0.95677889, 0.99759847],
       [0.60632077, 0.42066488, 0.05687041, 0.94658752, 0.65007123]])

In [23]:
list(range(2,10))

[2, 3, 4, 5, 6, 7, 8, 9]

In [26]:
for i,key in enumerate({i:i*2 for i in range(10)}): print(i,key)

0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9


In [120]:
np.argmin(np.array([3.,5.,6.]))

0

In [9]:
npar1=np.array([2,3])
npar2=np.array([[4,5],[6,7]])
npar1-npar2

array([[-2, -2],
       [-4, -4]])

In [7]:
def multi_index(nestedlist,indices,assignment=None):
    #Function that indexes an arbitrarily nested list by index sequence given by argument 'indices'.
    #Note that multiindexing is built into numpy arrays (via the syntax arr[(i,j,k)]) but not built in for possibly jagged python lists.
    #How to write this with lambdas (yes, there is a way to use them recuresively) instead of named definition:
    #return (lambda f,nestedlist,indices: f(f,nestedlist,indices))(lambda f,nestedlist,indices: f(f,nestedlist,indices[:-1])[indices[-1]] if len(indices)>1 else nestedlist[indices[0]], nestedlist , indices)
    try:
        if len(indices)>1:
            return multi_index(nestedlist[indices[0]],indices[1:],assignment=assignment)
        elif assignment is None: return nestedlist[indices[0]]
        else: nestedlist[indices[-1]]=assignment
        #return multi_index(nestedlist[0],indices[1:]) if len(indices)>1 else (nestedlist[indices[0]] if assignment is None else nestedlist[indices[0]]=assignment) #<-- invalid syntax because of the assignment, I believe.
    except IndexError as ie:
        print('Address incompatible with nested list')

In [8]:
class ClusterTree():
    '''
    Cluster Tree object maintains a map of the tree (leaves being the indices of the clusterized data) together with a set of 
    active addresses which point to branch nodes (clusters) of the tree. The active set can be expanded as deemed necessary by 
    successively higher resolution rbf-SVM training. For evaluation of the radius of clusters, the cluster tree also maintains 
    distance matrices between cluster centers of level l-1 and level l for all levels l>1 of the tree.
    Euclidean radii of level 1 clusters are also stored, in the attribute indexmap. For each level1 cluster center c, indexmap
    stores a list whose lth term (len(nlist)>=l>0) is the row index corresponding to c in the lth distance matrix.
    
    Idea for memory and speed optimization: make index map into a dictionary that stores, for each level1 center c, a list of cluster
    radii for c, when c is treated as the cluster center for a cluster at sucessively higher levels, starting at level 1. The radii
    are calculated in the clusterization process. The radius list length thus depends on c (only some centers are also centers for
    higher level clusters)
    
    Term 'center' appearing in the code of ClusterTree refers to a member of initialization argument 'indices' that is used to index the data.'''
    def __init__(self,nlist,indices):
        #data is represented by input argument 'indices', which could be a subset of the indices of the main data to be clusterized
        #nlist is the number of clusters at each level of the tree.
        assert (nlist==np.sort(nlist)[::-1]).all(), "Cluster numbers of ascending levels in tree must be decreasing"
        print('nlist: ',nlist)
        self.active_addresses_=[i for i in range(nlist[-1])] #list of active addresses represented by a keyless dictionary structure (jagged list) corresponding to a dictionary of the form {0:0,1:1,2:{0:0,1:1}} (corresponding to address list ['0','1','2.0','2.1'])
        self.tree_map=indices #diagram of cluster tree represented by nested list (to be nestified by clusterize(*args)) s.t. the data indices at level 0 is indexed by address (i_L,i_(L-1),...,i_1,i_0) where 0<=i_l<len(tree_map[i_L,...,i_(l+1)]) for all 0<=l<=L.
        self.nlist=nlist
        #self.distance_matrix=np.zeros((self.n2/2,2*self.n1-1-self.n2)) if self.n2%2 else np.zeros((self.n2,self.n1-1-self.n2//2)) #compactified euclidean distance matrix beetween level1 and level2 cluster centers to avoid storing zeros. <-- Depracated
        self.dist_matrices=[np.zeros((nlist[i+1],nlist[i]-nlist[i+1])) for i in range(len(nlist)-1)] #to access the row/column of a distance matrix at level_l(holding all distances to level_l+1) for a center c, use indexmap[c][l+1]/indexmap[c][l]
        self.indexmap={} #dictionary whose keys are to be centers corresponding to level1 clusters, and whose values are lists whose first element is the level1 radius, and whose following elements are indices used to index the distance matrices at successively higher levels.
        #more explanation on indexmap: named so becuase it maps the cluster c to its distance matrix index self.indexmap[c][l] for the lth-level distance matrix.
        self.data_loaded=False
    
    def clusterize(self,data,mus=None,sigmas=None,priors=None):
        #data is the full data from which the tree is initialized as the subset of indices self.tree_map
        #Optional gmm parameters mus, sigmas and priors can be given to better inform the choice of cluster centers
        assert len(data)>=np.max(self.tree_map), "Data incompatible with the tree's initialization indices"
        if mus!=None:
            #This block defines the probabobility parameter level1_center_dist for choosing cluster centers according to gmm distribution if paramters are given otherwise defines it as uniform. 
            #If gmm is an accurate model for the distribution of the data, a uniform choice probability will capture the correct distribution and this step is unnecessary
            assert len(mus)==len(sigmas) and len(mus)==len(priors), 'gmm parameters for cluster centers must have same length'
            x=data[self.tree_map]
            level1_center_distr=(lambda x:x/np.sum(x))(np.sum(np.fromiter(map(lambda params:(lambda sig,mu,p: p*1/np.sqrt((2*np.pi)**np.shape(data)[1]*np.linalg.det(sigma))*np.exp(-1/2*np.linalg.multi_dot([x-mu,np.linalg.inv(sig),(x-mu).T])))(*params),zip(sigmas,mus,priors)),dtype=float),axis=0))
        else: level1_center_distr=np.zeros(len(self.tree_map))+1/len(self.tree_map)
        cluster_center_distr=level1_center_distr
        def group(clusters,nnew,level): #given a set of clusters at level-1, group the clusters into nnew groups at current level
            nonlocal cluster_center_distr
            centers=np.fromiter(((x if type(x)!=list else multi_index(x,np.zeros(level-1,dtype=int))) for x in clusters),dtype=int) #representing centers by their indices in the initializiation data
            #print('centers of level '+str(level-1)+': ',centers)
            new_center_indices=np.random.choice(len(clusters),nnew,replace=False,p=cluster_center_distr) #a random sampling of range(len(center index list))
            new_centers=centers[new_center_indices]
            if level==1: 
                for nc in new_centers: self.indexmap[nc]=[0]
            if level>1: #added for evaluation of cluster radii from memory. Maps centers to a list of indices (with the first element being reserved for the level1 radius) whose i'th element is used as the index of the level i+1-to-i distance matrix.
                for i,c in enumerate(np.delete(centers,new_center_indices)):self.indexmap[c]+=[i]
                for i,nc in enumerate(new_centers):self.indexmap[nc]+=[i]
            #print('indexmap at level '+str(level)+':\n',self.indexmap)
            #print('number of indexmap keys:',len(self.indexmap.keys()))
            grouped_clusters=[[clusters[nci]] for nci in new_center_indices] #grouped clusters, after evaluation of assign_cluster on all non-new center points, becomes the input clusters with an added level of grouping at depth 1.
            #print('center indices:',end='')
            def assign_cluster(center_index,center): # assigns cluster with approximate center given by input variable center to the nearest cluster in grouped_clusters.
                #center_index is the index of center in the list centers (don't forget that center itself is an index of data)
                nonlocal new_centers,grouped_clusters
                dists_to_clusters=np.linalg.norm(data[center]-data[new_centers],axis=1) #array of distances of center to each cluster center
                #print('.',end='')
                minindex=np.argmin(dists_to_clusters) #0<=minindex<nnew
                mindist=dists_to_clusters[minindex]
                assigned_center=new_centers[minindex]
                #print('\nindexlist of assigned_center ('+str(assigned_center)+'):',self.indexmap[assigned_center])
                # potential_radius=sqrt((2-np.exp()) mindist+self.indexmap[assigned_center][level-1]
                # if potential_radius>self.indexmap[level]: self.indexmap[level]=potential_radius
                if level==1 and mindist>self.indexmap[assigned_center][0]: self.indexmap[assigned_center][0]=mindist #by the time assign_cluster has been called on all centers at level1, this line will have assigned the radii of level1 clusters.
                if level>1: self.dist_matrices[level-2][(np.arange(nnew),self.indexmap[center][level-1])]=dists_to_clusters
                grouped_clusters[minindex]+=[clusters[center_index]]
                #print(str(center_index)+' ', end='')
                #print('cluster '+str(center)+' assigned')
            start=time.time()
            nonnew_centers=np.delete(np.transpose([np.arange(len(centers)),centers]),new_center_indices,axis=0)
            for args in nonnew_centers: assign_cluster(*args) #executing this line with concurrent.futures does not catch exceptions. Use this line instead for debugging.
            #with statement below: parallel version of 'for c in nonnew_centers: assign_cluster(c)'
            #with concurrent.futures.ThreadPoolExecutor(max_workers=1) as exe: 
            #    exe.map(lambda args:assign_cluster(*args),nonnew_centers)
            print('\nTime elapsed for grouping of level '+str(level-1)+':',time.time()-start)
            cluster_center_distr=(lambda x:x/np.sum(x))(cluster_center_distr[new_center_indices])
            return grouped_clusters
        for l,n in enumerate(self.nlist,start=1):
            self.tree_map=group(self.tree_map,n,l)
            #print('tree at level '+str(l)+': ',self.tree_map)
        self.data_loaded=True

    def radius(self,add,data,gamma): #returns the radius of cluster with address add
        # Note that cluster tree is written to save memory, so it does not store the data itself, but only the clustering structure of the data. Hence why the data needs to be given as an argument to evaluate the radii of clusters.
        assert self.data_loaded, "Cluster tree has not been trained yet (use method clusterize)."
        treesize=(lambda f,jagger: f(f,jagger))(lambda f,jagger: np.sum(np.fromiter((f(f,lis) for lis in jagger),dtype=np.uintc)) if type(jagger[0]) is list else len(jagger),self.tree_map) #calculate number of leaves in the tree
        #print('size of data: ',len(data))
        #print('size of tree map: ',treesize) #This line and the one above it is to check that all data points have been incorporated into the indextree given be treemap.
        #print('tree map:\n',self.tree_map)
        assert len(data)>=treesize, "Data is incompatible with tree."#np.size(self.tree_map)
        def get_rad(address):
            route=[int(x) for x in address.split('.')]
            try: # evaluate radius using address
                if len(route)==len(self.nlist): return np.sqrt(2-2*np.exp(-gamma*self.indexmap[self.center(address)][0]**2))
                if len(route)==len(self.nlist)+1: return 0.
                subclusters=(address+'.'+str(k) for k in range(len(multi_index(self.tree_map,route)))) #representing subclusters by their addresses.
                #print('subclusters: ',[sub for sub in subclusters])
                return max((np.sqrt(self.memory_dist(address,sub,gamma=gamma)**2+get_rad(sub)**2) for sub in subclusters))
            except IndexError as ie:
                print('Address likely not compatible with tree: ',str(ie))
        return get_rad(add)
    
    def pad(self,address):
        x='.'.join(['0' for i in range(1+len(self.nlist)-len(address.split('.')))])
        #print(x)
        return address+bool(x)*'.'+x
    
    def center(self,address):#get index (in data) of center point of cluster whose address is given
        assert self.data_loaded, "Cluster tree has not been trained yet (use method clusterize)."
        route=[int(x) for x in self.pad(address).split('.')]
        return multi_index(self.tree_map,route)
                                     
    def memory_dist(self,ad1,ad2,gamma=0.1): #gets the distance between cluster centers at level1 from attribute self.distance_matrix
        #If ad1 does not point to a cluster at a level one higher than does ad2, this function will raise an exception.
        route1,route2=tuple(map(lambda ad: [int(x) for x in ad.split('.')],[ad1,ad2]))
        assert len(route1)+1==len(route2), 'The first address must point to a cluster at a level one higher than does the second address'
        if route2[-1]==0: return 0
        treelevel=len(self.nlist)+1-len(route2) #treelevel of the column index of the distance matrix
        try: i,j=tuple(map(lambda ad: self.indexmap[self.center(ad)][treelevel],[ad1,ad2]))
        except IndexError as ie: print('treelevel exceeds size of index list of center'+str(self.center(ad))+'\n',str(ie))
        euclidean_dist=self.dist_matrices[treelevel-1][(i,j)]
        return np.sqrt(2-2*np.exp(-gamma*euclidean_dist**2))

    def active_gen(self):
        #generator for iterating over the list of active addresses
        def address_gen(nestyboi):
            for i,key in enumerate(nestyboi):
                if type(key) is not list:
                    yield str(key)
                else: yield from (str(i)+'.'+leaf for leaf in address_gen(key))
        return address_gen(self.active_addresses_)
    
    def active_indices(self):
        retlist=[]
        for ad in self.active_gen():
            try: retlist+=[multi_index(self.tree_map,[int(x) for x in self.pad(ad).split('.')])]
            except ValueError as ve:
                print('Value error encountered: ', str(ve))
                print('address that generated error: ', ad)
                print('route associated with faulty address: ',self.pad(ad).split('.'))
                raise ve
        return np.array(retlist,dtype=int)
        #return np.array([multi_index(self.tree_map,[int(x) for x in self.pad(ad).split('.')]) for ad in self.active_gen()],dtype=int)
    
    def expand(self,ad): #returns the addresses of the non-center members of the expanded cluster
        route=[int(x) for x in ad.split('.')]
        assert len(route)<=len(self.nlist), 'Cluster address '+str(ad)+' is maximally resolved and cannot be expanded'
        cluster_at_ad=multi_index(self.active_addresses_,route)
        if type(cluster_at_ad)!=int:
            print('Warning: cluster with address '+str(ad)+' has children ('+str(len(cluster_at_ad))+' of them), indicating it is already expanded')
        ## for memory-efficient version ##
        nchildren=len(multi_index(self.tree_map,route))
        children=[x for x in range(nchildren)]
        multi_index(self.active_addresses_,route,assignment=children)
        return (ad+'.'+str(child) for child in children)
    
    def collapse(self,ad):
        route=[int(x) for x in ad.split('.')]
        multi_index(self.active_addresses_,route,assignment=route[-1])
        
        

In [140]:
[i for i in range(1,5)]

[1, 2, 3, 4]

In [13]:
def cluster_train_svm(data,labels,gamma=0.15,C=1):
    #data is a numpy array containing the feature vectors of the data
    pump_indices=pump_indices=np.random.choice(np.where(labels)[-1],600,replace=False) #np.where returns a tuple containing the selection array (this is to be consistent with the input array's dimension, so that the output of np.where can be used to multi_index an array)
    nopump_indices=np.random.choice(np.where(np.logical_not(labels))[-1],1000,replace=False)
    tree_height=7
    npumps=len(pump_indices)
    nnopumps=len(nopump_indices)
    print('# of pumps:',npumps)
    print('# of nopumps:',nnopumps)
    pump_nlist,nopump_nlist=tuple(map(lambda n:np.vectorize(int)(n//n**(np.arange(1,tree_height+1)/(tree_height+1))),[npumps,nnopumps]))
    pump_tree=ClusterTree(pump_nlist,pump_indices)
    print('\nClusterizing pumps')
    pump_tree.clusterize(data)
    nopump_tree=ClusterTree(nopump_nlist,nopump_indices)
    print('\nClusterizing nopumps')
    nopump_tree.clusterize(data)
    
    svc=SVC(C=C,kernel='rbf',gamma=gamma,class_weight='balanced',verbose=True,decision_function_shape='ovo')
    intersections_found=True
    unexpanded_count=0 #counter for number of unexpanded clusters
    while intersections_found:
        intersections_found=False
        active_pump_indices, active_nopump_indices=tuple(map(lambda tree:tree.active_indices(),[pump_tree,nopump_tree]))
        active_indices=np.concatenate((active_pump_indices,active_nopump_indices))
        nactive=len(active_indices)
        print('\n-----------------------------\n')
        print('Training svm for expanded tree')
        svc.fit(data[active_indices],labels[active_indices])
        #calcualate norm (wnorm) of the weight vector in the embedded rbf space based on \vec{w}=\sum_{supp.vec.s} \alpha_i*y_i*\phi(sv_i) where \phi is the embedding function
        print('\nEvaluating norm of weight vector')
        #wnorm=np.sqrt(np.sum([[(lambda arr:arr[(0,i)]*arr[(0,j)])(svc.dual_coef_)*np.e**(-gamma*np.linalg.norm(veci-vecj)**2) for i,veci in enumerate(svc.support_vectors_)] for j,vecj in enumerate(svc.support_vectors_)]))
        #-------------------------------
        # wnorm=(lambda v,svecs:np.sqrt(np.linalg.multi_dot([v,np.exp(-gamma*np.array(\
        #                                   [[np.linalg.norm(svi-svj)**2 for svi in svecs]\
        #                                                                for svj in svecs])),v])))(svc.dual_coef_[-1],svc.support_vectors_)
        #-------------------------------
        wnorm=(lambda v,svecs:np.sqrt(np.linalg.multi_dot([v,np.exp(-gamma*np.fromiter(\
                                     (np.linalg.norm(svi-svecs,axis=1)**2 for svi in svecs),dtype=np.dtype(str(len(svecs))+"f8")))\
                                                            ,v])))(svc.dual_coef_[-1],svc.support_vectors_)
        #dual_coef_ is a 2d row vector wit as many columns as support vectors, hence needs to be flattened ([-1])
        print('|w|='+str(wnorm))#,'; built-in |w|='+str(np.linalg.norm(svc.coef_)))#,str(wnorm2))
        nsupport=svc.n_support_ #2-element array of support vector counts in each class.
        print('# active centers: '+str((len(active_nopump_indices),len(active_pump_indices))), '# support vecs: '+str(nsupport))
        def expansion_step(tree,label): #input label is the label of the tree
            nonlocal intersections_found, unexpanded_count
            support_ind_indices=svc.support_
            support_indices=active_indices[support_ind_indices]
            nonsupport_indices=np.delete(active_indices,support_ind_indices)
            for ad in tree.active_gen():
                #print('ad: ',ad)
                try: subclusters=tree.expand(ad) #subclusters is a list of addresses of the subclusters
                except AssertionError as ae: 
                    print('\nCluster '+ad+' could not be expanded:\n',str(ae))
                    continue
                if np.sum(nsupport)==nactive:
                    intersections_found=True
                    continue
                print('\nchecking for border violations among subclusters of '+ad+' in '+bool((label+1)/2)*'pump'+(not (label+1)/2)*'nopump'+' tree')
                for sub in subclusters:
                    if tree.center(sub) in support_indices:
                        intersections_found=True
                        #print(sub+' is a support vector - the cluster with address '+sub[:-2]+' will be expanded.')
                        break
                    radius=tree.radius(sub,data,gamma)
                    decision_val=svc.decision_function([data[tree.center(sub)]])
                    #print('Manually evaluating decision value of cluster center '+sub)
                    #manual_decision_val=np.sum(svc.dual_coef_[-1]*np.exp(-gamma*np.linalg.norm(data[tree.center(sub)]-svc.support_vectors_,axis=1)**2))+svc.intercept_
                    if radius + 1/wnorm >= label*decision_val/wnorm: #the decision function for a test vector x i is w.x+b
                        intersections_found=True
                        print('First subcluster found to intersect the separatrix:',sub, '\nradius: '+str(radius)+\
                              ';\ndecision value: '+str(decision_val),'\ndistance from separatrix: '+str(decision_val/wnorm))#+';\nmanual decision value: '+str(manual_decision_val))
                        break
                if not intersections_found:
                    unexpanded_count+=1
                    print('\nno intersections found for cluster '+ad+'!')
                    tree.collapse(ad)
        expansion_step(pump_tree,1)
        expansion_step(nopump_tree,-1)
        #(expansion_step(*params) for params in [[pump_tree,1],[nopump_tree,-1]])
    print('\nCB-SVM training complete. Number of unexpanded clusters: ',str(unexpanded_count))
    return (svc,pump_tree,nopump_tree)
    

In [34]:
nopump_tree.active_addresses_

[[[[[[[0, 1, 2], [0, 1, 2], [0, 1], [0]],
     [[0], [0], [0, 1, 2], [0, 1], [0, 1, 2, 3, 4, 5], [0]],
     [[0]]],
    [[[0], [0, 1, 2]], [[0], [0]], [[0, 1, 2], [0]]],
    [[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
      [0, 1],
      [0, 1, 2, 3, 4],
      [0, 1],
      [0, 1, 2, 3, 4],
      [0],
      [0]],
     [[0, 1], [0]],
     [[0, 1], [0, 1]],
     [[0]],
     [[0, 1, 2, 3]],
     [[0]],
     [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
      [0],
      [0, 1, 2],
      [0],
      [0],
      [0, 1, 2, 3],
      [0],
      [0, 1],
      [0],
      [0, 1]],
     [[0, 1]],
     [[0, 1, 2, 3, 4, 5],
      [0],
      [0],
      [0, 1, 2],
      [0],
      [0],
      [0, 1, 2, 3, 4, 5],
      [0, 1],
      [0],
      [0],
      [0, 1, 2, 3, 4, 5],
      [0],
      [0],
      [0, 1]]],
    [[[0]], [[0, 1]], [[0, 1]]]]],
  [[[[[0]], [[0], [0]], [[0, 1], [0], [0, 1], [0]], [[0, 1]]]],
   [[[[0, 1, 2], [0], [0, 1]]]]],
  [[[[[0], [0, 1, 2, 3, 4]]], [[[0]]], [[[0]], [[0]]]]],
  [[[[[0, 1, 2, 3, 4], [0,

In [399]:
svc.support_vectors_.dtype

dtype('float64')

In [413]:
np.dtype('10687<f8')

dtype(('<f8', (10687,)))

In [3]:
iterable=((x,np.sqrt(x**3)) for x in range(5))
np.fromiter(iterable,dtype=np.dtype("2f8"))

array([[0.        , 0.        ],
       [1.        , 1.        ],
       [2.        , 2.82842712],
       [3.        , 5.19615242],
       [4.        , 8.        ]])

In [11]:
def trial(svecs):
    print((np.linalg.norm(svecs[1]-svecs,axis=1)**2).shape)
    iterable=(np.linalg.norm(svi-svecs,axis=1)**2 for svi in svecs)
    
    return np.fromiter(iterable,dtype=np.dtype(str(len(svecs))+"<f8"))
trial(svc.support_vectors_)

(1252,)


array([[  0.        ,  16.49750173,  31.43395484, ..., 152.52098482,
        144.00232326, 174.34326168],
       [ 16.49750173,   0.        ,  28.62433507, ..., 157.62077174,
        146.73059177, 157.89501054],
       [ 31.43395484,  28.62433507,   0.        , ..., 162.53619949,
        150.77182285, 158.14562207],
       ...,
       [152.52098482, 157.62077174, 162.53619949, ...,   0.        ,
         30.68421155,  41.34899237],
       [144.00232326, 146.73059177, 150.77182285, ...,  30.68421155,
          0.        ,  58.29598183],
       [174.34326168, 157.89501054, 158.14562207, ...,  41.34899237,
         58.29598183,   0.        ]])

In [ ]:
np.fromiter(((x+y for x in range(2)) for y in range(5,7)),

In [294]:
np.delete([34,5,7],1)

array([34,  7])

In [186]:
help(SVC.decision_function)

Help on function decision_function in module sklearn.svm._base:

decision_function(self, X)
    Evaluate the decision function for the samples in X.
    
    Parameters
    ----------
    X : array-like of shape (n_samples, n_features)
        The input samples.
    
    Returns
    -------
    X : ndarray of shape (n_samples, n_classes * (n_classes-1) / 2)
        Returns the decision function of the sample for each class
        in the model.
        If decision_function_shape='ovr', the shape is (n_samples,
        n_classes).
    
    Notes
    -----
    If decision_function_shape='ovo', the function values are proportional
    to the distance of the samples X to the separating hyperplane. If the
    exact distances are required, divide the function values by the norm of
    the weight vector (``coef_``). See also `this question
    <https://stats.stackexchange.com/questions/14876/
    interpreting-distance-from-hyperplane-in-svm>`_ for further details.
    If decision_function_sha

In [259]:
(2*v)**2

array([4., 4., 4., 4., 4., 4., 4., 4., 4., 4.])

In [247]:
v=np.ones((1,10))[-1]
M=np.random.rand(10,10)
np.linalg.multi_dot([v,M,v])

51.02906131583193

In [200]:
np.exp(-2*np.linalg.norm(np.array([[4,5]])-np.array([[5,6],[6,7]]).T,axis=1))

array([0.05910575, 0.00349349])

In [203]:
np.e**1

2.718281828459045

In [267]:
len(nopump_tree.active_indices())

500

In [124]:
multi_index(nopump_tree.active_addresses_,[int(x) for x in '1.3.3.5.1.2'.split('.')])

[0, 1, 2]

In [167]:
np.where(np.array([[4,2],[4,5]]) == np.array([4,5]))

(array([0, 1, 1], dtype=int64), array([0, 0, 1], dtype=int64))

In [168]:
np.array([[4,2],[4,5]]) == np.array([4,5])

array([[ True, False],
       [ True,  True]])

In [291]:
#evaluate svc below before this block
support_indices=svc.support_

svc.decision_function(data[support_indices])#data[np.random.choice(pump_tree.active_indices(),100)])
support_coeffs=svc.dual_coef_

In [253]:
len(np.where(support_coeffs[-1]<0)[-1])

362

In [293]:
len(support_indices)

761

In [292]:
support_coeffs[-1]

array([-1.5       , -1.5       , -0.94166273, -1.5       , -1.5       ,
       -1.5       , -1.5       , -1.5       , -0.2390507 , -1.5       ,
       -1.5       , -1.5       , -1.5       , -1.0164395 , -1.5       ,
       -1.5       , -1.5       , -1.5       , -1.18828139, -1.5       ,
       -1.5       , -1.5       , -1.5       , -1.5       , -1.5       ,
       -1.5       , -1.5       , -1.5       , -1.5       , -1.5       ,
       -1.5       , -1.5       , -1.5       , -1.5       , -1.5       ,
       -1.5       , -1.5       , -1.5       , -1.5       , -1.5       ,
       -1.5       , -1.5       , -1.5       , -1.5       , -1.5       ,
       -1.5       , -1.5       , -1.5       , -1.5       , -1.5       ,
       -1.5       , -1.5       , -1.5       , -1.5       , -1.5       ,
       -1.5       , -1.5       , -1.5       , -1.5       , -1.5       ,
       -1.5       , -1.5       , -1.5       , -1.5       , -1.5       ,
       -1.5       , -1.5       , -1.5       , -1.5       , -1.5 

In [47]:
pump_tree.dist_matrices[4]

array([[14.61062302, 16.15296466, 12.32863325, 13.75199116, 13.88073148],
       [11.38209321,  8.95254151, 14.37020959, 12.87453772, 12.94426626]])

In [90]:
pump_tree.nlist

array([372, 138,  51,  19,   7,   2])

In [92]:
[val[0] for val in pump_tree.indexmap.values()] #Level one radii

[0,
 0,
 6.396339569031158,
 0,
 0,
 4.536958228289314,
 7.297900261834267,
 5.631372320584118,
 0,
 6.363528218981475,
 0,
 0,
 4.83319814650992,
 5.4910207866256995,
 0,
 2.870709082388217,
 6.332203826799228,
 0,
 0,
 6.892847152827254,
 2.7809530151814306,
 0,
 0,
 6.934106628280604,
 5.962232703739505,
 6.722773584955508,
 5.185465529181254,
 0,
 4.664927614180773,
 8.59471360085512,
 7.562154253208724,
 5.709315503657375,
 0,
 0,
 0,
 6.000391857213247,
 4.097365190466732,
 4.6422729480663385,
 4.141517229864331,
 0,
 6.059738283455672,
 0,
 6.591840601231617,
 5.367455712988739,
 5.840678858373343,
 0,
 6.3980443361068655,
 0,
 4.14536555959023,
 4.740454711542265,
 0,
 5.964656242270527,
 5.572750794059963,
 0,
 6.905318406632668,
 4.319167732685954,
 4.659771888411441,
 7.655718938603679,
 5.673564621374046,
 3.6381751225659746,
 7.697131248355211,
 0,
 7.26102094446751,
 4.668810723958001,
 0,
 4.2298198792052,
 0,
 5.74590148920682,
 0,
 5.081415436178045,
 6.607650099713822

In [129]:
[4,5,6][:-1]

[4, 5]

In [134]:
#check what percentage of radii at mid level clustering are zero
for i in range(2):
    for ad in pump_tree.active_gen():
        pump_tree.collapse('.'.join(ad.split('.')[:-2]))
mid_level_radii=np.fromiter((pump_tree.radius(ad,data,0.15) for ad in pump_tree.active_gen()),dtype=float)
print(len(mid_level_radii))
print(len(np.where(mid_level_radii)))

ValueError: invalid literal for int() with base 10: ''

In [98]:
len(np.where(np.array([val[0] for val in pump_tree.indexmap.values()])==0.)[-1])

247

In [349]:
svc.dual_coef_

array([[-80.        , -80.        , -46.46659897, -80.        ,
        -36.97391221, -80.        , -80.        , -57.94121699,
        -80.        , -22.87754738, -48.30920422, -80.        ,
        -50.02201947, -80.        , -80.        , -63.11790391,
        -34.45155862, -80.        , -80.        , -80.        ,
         -9.55029931, -80.        , -60.71520884, -80.        ,
        -80.        , -80.        , -80.        , -80.        ,
        -80.        , -15.1776152 , -36.22811777, -63.04963092,
        -69.83363686, -80.        ,  -2.8443054 , -47.99003201,
        -80.        ,  -4.27749261, -80.        , -80.        ,
        -75.08009822, -80.        , -77.13230971, -80.        ,
        -80.        , -80.        , -56.41828472, -35.88232534,
        -52.79948252, -65.90154479, -80.        , -80.        ,
        -80.        , -56.07314875, -80.        , -80.        ,
        -80.        , -48.13857836, -80.        , -80.        ,
        -69.01847641, -80.        , -45.

In [14]:
svc,pump_tree,nopump_tree=cluster_train_svm(data,labels,gamma=8e-4,C=1.56)

# of pumps: 600
# of nopumps: 1000
nlist:  [269 121  54  24  11   4   2]

Clusterizing pumps

Time elapsed for grouping of level 0: 0.029918909072875977

Time elapsed for grouping of level 1: 0.013045787811279297

Time elapsed for grouping of level 2: 0.003466367721557617

Time elapsed for grouping of level 3: 0.0009975433349609375

Time elapsed for grouping of level 4: 0.0011000633239746094

Time elapsed for grouping of level 5: 0.0

Time elapsed for grouping of level 6: 0.0008969306945800781
nlist:  [421 177  74  31  13   5   2]

Clusterizing nopumps

Time elapsed for grouping of level 0: 0.09175419807434082

Time elapsed for grouping of level 1: 0.013961553573608398

Time elapsed for grouping of level 2: 0.003987312316894531

Time elapsed for grouping of level 3: 0.0019953250885009766

Time elapsed for grouping of level 4: 0.000997781753540039

Time elapsed for grouping of level 5: 0.0

Time elapsed for grouping of level 6: 0.0

-----------------------------

Training svm for expand

In [25]:
len(pump_tree.active_indices())

1000

In [26]:
pump_tree.active_indices()

array([ 3383, 23281, 44494,  3755, 15502, 41267, 26226, 21952,  1688,
       20082, 48730, 35032, 24458, 33174, 19113,  7139, 45832, 28986,
       45535, 28893, 48706, 26864, 28249, 26184,  2279, 14203, 42398,
       39221,  5680, 31624, 29773, 30034,  1686,  3541,   481,  4693,
       24626, 30879,  5141, 19786, 14131, 34064, 36000, 42339, 10759,
       16702, 22569, 40207, 34482, 23140, 38576, 12751, 24239, 21027,
       28050, 44631,  2581,  2235, 30907, 49215, 30224, 44252, 38072,
       26452, 48348, 40067,  2702, 23983,  9260, 48418,  5718, 18338,
       34177, 19506, 30908, 45522, 36978, 41224, 32853, 43869, 23785,
        6456, 45894,  8172,  7797,  5823, 51226,  4213, 47443, 39985,
       44849, 20032,  7356, 17495, 24872, 42319, 28713, 27200, 18148,
       47830, 39946, 14281, 28415, 20189, 24628, 48373, 42285, 23349,
       32509, 28023, 24543,  1732, 37029, 34081, 23307,  3958, 20527,
       12904, 32016, 45654,  9773, 44901, 47098,  1384, 35516, 25405,
       21223, 54853,

In [7]:
np.linalg.norm(np.array([3,4])-np.array([[5,6],[1,3]]),axis=1)

array([2.82842712, 2.23606798])

In [145]:
'a'.split('.')

['a']

In [152]:
pump_indices=np.random.choice(np.where(labels)[-1],1000,replace=False) #np.where returns a tuple containing the selection array (this is to be consistent with the input array's dimension)
nopump_indices=np.where(np.logical_not(labels))[-1]
tree_height=6
npumps=len(pump_indices)
nnopumps=len(nopump_indices)
print('# of pumps:',npumps)
print('# of nopumps:',nnopumps)
pump_nlist,nopump_nlist=tuple(map(lambda n:np.vectorize(int)(n//n**(np.arange(1,tree_height+1)/(tree_height+1))),[npumps,nnopumps]))
pump_tree=ClusterTree(pump_nlist,pump_indices)
print('Clusterizing pumps')
pump_tree.clusterize(data)
'''
nopump_tree=ClusterTree(nopump_nlist,nopump_indices)
print('Clusterizing nopumps')
nopump_tree.clusterize(data)'''

# of pumps: 1000
# of nopumps: 29587
nlist:  [372 138  51  19   7   2]
Clusterizing pumps

Time elapsed for grouping of level 0: 0.10574531555175781

Time elapsed for grouping of level 1: 0.021394968032836914

Time elapsed for grouping of level 2: 0.007036685943603516

Time elapsed for grouping of level 3: 0.00293731689453125

Time elapsed for grouping of level 4: 0.00040602684020996094

Time elapsed for grouping of level 5: 0.0


"\nnopump_tree=ClusterTree(nopump_nlist,nopump_indices)\nprint('Clusterizing nopumps')\nnopump_tree.clusterize(data)"

In [154]:
[multi_index(pump_tree.tree_map,[int(x) for x in pump_tree.pad(ad).split('.')]) for ad in pump_tree.active_gen()]

0.0.0.0.0.0
0.0.0.0.0.0


[14488, 19990]

In [153]:
pump_tree.active_indices()

0.0.0.0.0.0
0.0.0.0.0.0


array([14488, 19990])

In [136]:
(lambda f,jagger: f(f,jagger))(lambda f,jagger: np.sum(np.fromiter((f(f,lis) for lis in jagger),dtype=np.uintc)) if type(jagger[0]) is list else len(jagger),pump_tree.tree_map)

100

In [137]:
def flatten(jagger):
    for lis in jagger:
        if type(lis) is not list: yield lis
        else: yield from flatten(lis)
len({x for x in flatten(pump_tree.tree_map)})

100

In [22]:
svc=SVC(C=1,gamma=0.02,class_weight='balanced',verbose=True)
pump_indices=np.random.choice(np.where(labels)[-1],1000,replace=False)
nopump_indices=np.random.choice(np.where(np.logical_not(labels))[-1],10000,replace=False)
chosen_indices=np.concatenate((pump_indices,nopump_indices))
svc.fit(data[chosen_indices],labels[chosen_indices])

[LibSVM]

SVC(C=1, class_weight='balanced', gamma=0.02, verbose=True)

In [180]:
svc.get_params()

{'C': 1,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': 'balanced',
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 0.15,
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': True}

In [24]:
np.sum(svc.decision_function(data[np.random.choice(np.where(labels)[-1],300)]))

58.47773936129087

In [198]:
len(np.where(svc.dual_coef_<0)[-1])

22450

In [199]:
np.shape(svc.dual_coef_)

(1, 43223)

In [200]:
len(data)

55141

In [50]:
assert np.size([[2,3,4],[5,6,1]])==6, 'error'

In [ ]:
pump_tree.active_indices()

0.0.0.0.0.0
0.0.0.0.0.0
0.0.0.0.0.0
0.0.0.0.0.0


array([18252, 44645, 40921, 50189])

In [165]:
for ad in pump_tree.active_gen():
    print(ad)

0
1
2
3


Miscellaneous/UnusedCode

In [ ]:
#self.data=pd.DataFrame(columns=['id'+str(i) for i in range(len(nlist))][::-1]).astype('category',copy=False) <-- later deemed to be a superfluous attribute
#self.data=self.data.reindex(index=range(len(data))) <-- command associated with attribute self.data in initial verison of ClusterTree/
#from initial version of ClusterTree().expand:
#self.active_addresses_.pop(ad)
#self.active_addresses_.update([ad+'.'+str(i) for i in range(len(multi_index(self.tree_map,route)))])


In [ ]:
bkm(nopumps)# kmaeans clustering

In [ ]:
bkm1=bkm(n_clusters=40000,random_state=0)
clusters=bkm1.fit_predict(nopumps)

KeyboardInterrupt: 

In [ ]:
concurrent.futures.ThreadPoolExecutor

In [ ]:
    ''' Depracated methods of ClusterTree class
    def dist(self,ad1,ad2,data,gamma=0.1):
        #assert len(data)==np.size(self.tree_map), "Data is incompatible with tree." <-- self.radius is the only method that calls dist, so this check is unnecessary (already checked in self.radius)
        c_i1=self.center(ad1)
        c_i2=self.center(ad2)
        c1=data[c_i1] #actual center point (vector in data space)
        c2=data[c_i2] 
        return np.sqrt(2-2*np.exp(-gamma*np.linalg.norm(c1-c2)))
    
    def convert_index(self, i_py,j_py): #Peform appropriate conversion to access compactified distance matrix for clusters 1<=i<=n2(in level2) and 1<=j<=n1(in level1)
        # The input and returned indices are in python format (starting from 0)
        # this method can be vectorized such that inputting an array into one of the arrguments results in a tuple of arrays that can be used to index numpy arrays.
        assert j>i, 'Column index must be greater than row index'
        i,j=(i_py+1,j-py+1)
        if self.n2%2: ip,jp= (self.n2+1-i,self.n1+1-j) if i>self.n2/2 else (i,self.n1-1+self.n2+j)
        else: ip,jp= (self.n2+1-i,self.n2//2+2-j) if j<=self.n2//2+1 else (i,j-self.n2//2-1)
        return (ip-1,jp-1)
    '''         